In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
import random

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


In [3]:
#Wyświetlam sobie interakcje dla konkretnego usera(dokładnie z id = 1), by sprawdzić działanie prepare_users
display(HTML(interactions_df.loc[interactions_df['user_id'].isin([1])].head(50).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
94,1,41,OffSeason,[2-3],Standard,[160-260],[2-2],True
95,1,41,OffSeason,[2-3],Standard,[160-260],[2-2],True
448,1,57,WinterVacation,[2-3],Standard,[160-260],[2-2],True
684,1,108,WinterVacation,[2-3],Nonref,[160-260],[2-2],True
1598,1,158,LowSeason,[4-7],Nonref,[160-260],[2-2],True
1968,1,308,LowSeason,[4-7],Nonref,[260-360],[5-inf],False
2027,1,160,OffSeason,[4-7],Nonref,[160-260],[3-4],False
3175,1,379,HighSeason,[2-3],Nonref,[260-360],[3-4],False
4470,1,103,OffSeason,[2-3],Nonref,[160-260],[2-2],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several version of the prepare_users_df method and test which works best in your recommender.

In [4]:
def prepare_users_df(interactions_df):

    # Write your code here
    
    #Implementacja users_df 
    users_df = interactions_df.add_prefix('User_')#Dodaję prefix, by odróżnić userów od itemów
    users_df.rename(columns={'User_user_id': 'user_id'}, inplace=True)#Zmieniam nazwę kolumny User_user_id na user_id, żeby zachować zgodność z nazewnictwem w dalszej części kodu(w rekomenderze)
    users_df=pd.get_dummies(users_df, columns=['User_term', 'User_length_of_stay_bucket', 'User_rate_plan', 'User_room_segment', 'User_n_people_bucket', 'User_weekend_stay'])#One-hot encoding przy pomocy get_dummies dla wszystkich kolumn
    users_df = users_df.drop('User_item_id', axis=1)#User_item_id jest niepotrzebne w userach, więc je usuwam

    users_df = users_df.groupby("user_id")#Grupowanie, żeby usunąć duplikaty
    users_df = users_df.max()#Max z każdej kolumny, 1 gdy user kiedykolwiek skorzystał z danego features, 0 gdy nigdy
    users_df = users_df.reset_index()
    
    
    #Implementacja user_features      
    user_features = list(users_df.columns.values)#Nazwy kolumn jako lista  
    user_features.pop(0)#usuwam user_id z features
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

['User_term_WinterVacation', 'User_term_Easter', 'User_term_OffSeason', 'User_term_HighSeason', 'User_term_LowSeason', 'User_term_MayLongWeekend', 'User_term_NewYear', 'User_term_Christmas', 'User_length_of_stay_bucket_[0-1]', 'User_length_of_stay_bucket_[2-3]', 'User_length_of_stay_bucket_[4-7]', 'User_length_of_stay_bucket_[8-inf]', 'User_rate_plan_Standard', 'User_rate_plan_Nonref', 'User_room_segment_[0-160]', 'User_room_segment_[160-260]', 'User_room_segment_[260-360]', 'User_room_segment_[360-500]', 'User_room_segment_[500-900]', 'User_n_people_bucket_[1-1]', 'User_n_people_bucket_[2-2]', 'User_n_people_bucket_[3-4]', 'User_n_people_bucket_[5-inf]', 'User_weekend_stay_True', 'User_weekend_stay_False']


,user_id,User_term_WinterVacation,User_term_Easter,User_term_OffSeason,User_term_HighSeason,User_term_LowSeason,User_term_MayLongWeekend,User_term_NewYear,User_term_Christmas,User_length_of_stay_bucket_[0-1],User_length_of_stay_bucket_[2-3],User_length_of_stay_bucket_[4-7],User_length_of_stay_bucket_[8-inf],User_rate_plan_Standard,User_rate_plan_Nonref,User_room_segment_[0-160],User_room_segment_[160-260],User_room_segment_[260-360],User_room_segment_[360-500],User_room_segment_[500-900],User_n_people_bucket_[1-1],User_n_people_bucket_[2-2],User_n_people_bucket_[3-4],User_n_people_bucket_[5-inf],User_weekend_stay_True,User_weekend_stay_False
0,1,1,0,1,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,1
47,50,1,0,1,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0,0,0,1,1,1,1,1
92,96,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1
111,115,1,0,1,0,0,0,0,0,1,1,1,0,1,0,0,1,1,0,0,1,1,1,1,1,1
675,706,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1
1699,1736,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,1,1
7639,7779,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,1,1,1


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into on-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because of the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features.

In [5]:
def prepare_items_df(interactions_df):
    
    # Write your code here
    #Podsumowując, proszę w metodzie prepare_items_df tworzyć features jedynie w oparciu o wektor opisujący dany item, tzn.:
    #term, length_of_stay_bucket, rate_plan, room_segment, n_people_bucket, weekend_stay.

    #Implementacja items_df  
    
    items_df = interactions_df.add_prefix('Item_')#Dodaję prefix, by odróżnić itemy od userów
    items_df.rename(columns={'Item_item_id': 'item_id'}, inplace=True)#Zmieniam nazwę kolumny Item_item_id na item_id, żeby zachować zgodność z nazewnictwem w dalszej części kodu(w rekomenderze)
    items_df=pd.get_dummies(items_df, columns=['Item_term', 'Item_length_of_stay_bucket', 'Item_rate_plan', 'Item_room_segment', 'Item_n_people_bucket', 'Item_weekend_stay'])#One-hot encoding przy pomocy get_dummies dla wszystkich kolumn wspomnianych na teamsach 
    items_df = items_df.drop('Item_user_id', axis=1)#Item_user_id jest niepotrzebne w itemsach, więc je usuwam

    items_df = items_df.groupby("item_id")#Grupowanie, żeby usunąć duplikaty
    items_df = items_df.max()#Max z każdej kolumny, 1 gdy dany features oferuje coś, 0 gdy nie
    items_df = items_df.reset_index()
    
    
    #Implementacja items_features      
    item_features = list(items_df.columns.values)#Nazwy kolumn jako lista    
    item_features.pop(0)#usuwam item_id z features

    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

['Item_term_WinterVacation', 'Item_term_Easter', 'Item_term_OffSeason', 'Item_term_HighSeason', 'Item_term_LowSeason', 'Item_term_MayLongWeekend', 'Item_term_NewYear', 'Item_term_Christmas', 'Item_length_of_stay_bucket_[0-1]', 'Item_length_of_stay_bucket_[2-3]', 'Item_length_of_stay_bucket_[4-7]', 'Item_length_of_stay_bucket_[8-inf]', 'Item_rate_plan_Standard', 'Item_rate_plan_Nonref', 'Item_room_segment_[0-160]', 'Item_room_segment_[160-260]', 'Item_room_segment_[260-360]', 'Item_room_segment_[360-500]', 'Item_room_segment_[500-900]', 'Item_n_people_bucket_[1-1]', 'Item_n_people_bucket_[2-2]', 'Item_n_people_bucket_[3-4]', 'Item_n_people_bucket_[5-inf]', 'Item_weekend_stay_True', 'Item_weekend_stay_False']


,item_id,Item_term_WinterVacation,Item_term_Easter,Item_term_OffSeason,Item_term_HighSeason,Item_term_LowSeason,Item_term_MayLongWeekend,Item_term_NewYear,Item_term_Christmas,Item_length_of_stay_bucket_[0-1],Item_length_of_stay_bucket_[2-3],Item_length_of_stay_bucket_[4-7],Item_length_of_stay_bucket_[8-inf],Item_rate_plan_Standard,Item_rate_plan_Nonref,Item_room_segment_[0-160],Item_room_segment_[160-260],Item_room_segment_[260-360],Item_room_segment_[360-500],Item_room_segment_[500-900],Item_n_people_bucket_[1-1],Item_n_people_bucket_[2-2],Item_n_people_bucket_[3-4],Item_n_people_bucket_[5-inf],Item_weekend_stay_True,Item_weekend_stay_False
0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object for later use in the recommend method. Overwrite the users_df variable. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real interactions and equal to 0 for those newly added interactions. Generate several negative interactions per every positive interactions (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and use initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, Neural Networks or any model of your choice.
  - A different approach where you treat each item as a class, you train directly on categorical features of items and users (you would have to design appropriate categorical features for users) and you fit classifiers (e.g. Decision Tree classifier, Naive Bayes classifier etc.) instead of regressors.

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]     
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]

        interactions_df.loc[:, 'interacted'] = 1
        negative_interactions = []

        # Write your code here
        
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.

        interactions_len = len(interactions_df) #Ilość interakcji użytkownik-item

        number_of_negative_interactions = self.n_neg_per_pos*interactions_len #Rozmiar, tzn. ile ma być tych krotek/tupli
        #print(number_of_negative_interactions)

        while len(negative_interactions) <= number_of_negative_interactions: #Póki lista nie osiągnie zadanego rozmiaru
            user = random.randint(0,interactions_len) #Generuje randomowy nr usera 
            item = random.randint(0,interactions_len) #Generuje randomowy nr itemu
            
            user_tmp = interactions_df['user_id'] == user
            item_tmp = interactions_df['item_id'] == item #Przypisuje wylosowane nr do tymczasowego użytkownika i itemu(tzn. do konkretnego usera i itemu z df)
            
            if (user_tmp != item_tmp).any(): #sprawdzam, czy istnieje interkacja pomiędzy userem i itemem
                negative_interactions.append((user, item, 0)) #jesli nie, dodaję krotke do listy negative_interactions
            #print(len(negative_interactions))

        interactions_df = pd.concat(
                    [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # Write your code here
        # Prepare users_df and items_df
        # For users_df you just need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations)
        # For items you have to apply the prepare_items_df method to items_df.
        
        users_df = pd.merge(self.users_df,users_df) #Merge self do users_df
        items_df = prepare_items_df(items_df) #Prepare_items dla items_df
        
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            
            # Write your code here
            # Create a Carthesian product of users from users_df and items from items_df
            
            dataList = list(items_df) #Konwersja z tupli na listę

            items_df = pd.DataFrame(dataList, columns=['item_id']) #Do dataframe
            users_df['key'] = 1 #Tymczasowe klucze
            items_df['key'] = 1 #Tymczasowe klucze

            carthesian = pd.merge(users_df, items_df, on ='key').drop("key", 1) #Produkt kartezjański
            

            # Write your code here
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            
            predict = self.model.predict(int(carthesian)) #Tu coś nie poszło... Próbowałam różnych sposobów, zmiany na str, float
            
            
            # Write your code here
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            
            
            chosen_ids = predict.sort_values('scores',ascending=False) #Sortowanie od największego do najmniejszego wyniku
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate) 


# Quick test of the recommender

In [11]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

print(items_df)



       item_id            term length_of_stay_bucket rate_plan room_segment  \
0            0  WinterVacation                 [2-3]  Standard    [260-360]   
1            1  WinterVacation                 [2-3]  Standard    [160-260]   
2            2  WinterVacation                 [2-3]  Standard    [160-260]   
3            3  WinterVacation                 [4-7]  Standard    [160-260]   
4            4  WinterVacation                 [4-7]  Standard      [0-160]   
...        ...             ...                   ...       ...          ...   
15897      763      HighSeason                 [2-3]    Nonref      [0-160]   
15985      764       OffSeason               [8-inf]  Standard      [0-160]   
16008      765       Christmas               [8-inf]  Standard      [0-160]   
16018      766         NewYear                 [4-7]  Standard    [160-260]   
16023      767         NewYear                 [4-7]    Nonref    [260-360]   

      n_people_bucket weekend_stay  
0             

In [12]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [13]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'DataFrame'

# Tuning method

In [ ]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(HTML(results.to_html()))
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [ ]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 7})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(linear_cbui_tts_results.to_html()))

In [ ]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(amazon_tts_results.to_html()))

In [ ]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(HTML(tts_results.to_html()))